In [2]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# data preprocess
target_stock = "3105"
threshold = 0
stock_df = pd.read_csv(r'C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\data\3105 2023.csv', encoding='utf-8', encoding_errors= 'ignore')
five_day_min_low = talib.MIN(stock_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(stock_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點
stock_df['five_day_min_low'] = five_day_min_low
stock_df['five_day_max_high'] = five_day_max_high
stock_df = stock_df[25:]
predicted_price_df = pd.read_csv(r"C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\predicted_result\CNN_RNN 3105 2023 predicted prices.csv", encoding='utf-8', encoding_errors = 'ignore')
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.index = stock_df['Date']
stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
stock_df['true_price'] = predicted_price_df['true_price'].values

# long short condition
stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
stock_df['actual_price_up'] = (stock_df['Close'] > stock_df['Close'].shift(1)).astype(int)
stock_df['long_condition'] = (stock_df['true_price'].shift(1)<=stock_df['predicted_price']) & ((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1) >= threshold)
stock_df['short_condition'] = (stock_df['true_price'].shift(1)>stock_df['predicted_price']) & (abs((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1)) >= threshold)
stock_df['long_condition'] = stock_df['long_condition'].astype(int)
stock_df['short_condition'] = stock_df['short_condition'].astype(int)

stock_df

,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transcation,buy_Dealer_Hedging,...,sell_Foreign_Investor,sell_Investment_Trust,five_day_min_low,five_day_max_high,predicted_price,true_price,predicted_price_up,actual_price_up,long_condition,short_condition
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-17,2023-02-17,5190000,896464000,169.5,175.0,169.0,173.0,2.0,3871,75000,...,2097572,0,166.0,176.5,0.291340,0.296817,0,0,0,0
2023-02-20,2023-02-20,4774000,843105000,174.0,178.5,174.0,178.0,5.0,4171,198663,...,867000,48000,168.5,178.5,0.296089,0.309154,1,1,0,0
2023-02-21,2023-02-21,6306000,1129913000,179.0,182.5,176.0,177.0,-1.0,5381,242184,...,2028000,2000,169.0,182.5,0.307615,0.306686,1,0,0,0
2023-02-22,2023-02-22,4362000,755355000,175.0,176.0,171.0,172.0,-5.0,3897,26000,...,1417000,0,169.0,182.5,0.308736,0.294350,1,0,0,0
2023-02-23,2023-02-23,2810000,489955000,174.0,176.0,172.0,174.5,2.5,2350,90951,...,1101296,2000,169.0,182.5,0.301502,0.300518,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25,2023-12-25,2853000,448995000,155.0,159.0,155.0,157.5,3.0,2554,120187,...,684000,3806,150.0,161.0,0.255213,0.258574,1,1,1,0
2023-12-26,2023-12-26,1945000,307613000,158.0,159.0,156.5,158.5,1.0,1978,46207,...,339510,3339,150.0,159.0,0.262403,0.261041,1,1,1,0
2023-12-27,2023-12-27,1722000,273991000,159.5,160.5,157.5,159.5,1.0,1954,33378,...,306132,266972,150.0,160.5,0.265496,0.263508,1,1,1,0


strategy

In [3]:
# long strategy
signals_long = np.zeros(len(stock_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(stock_df)):
    if partition_size_long == 0:
        if stock_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = stock_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (stock_df['Close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(stock_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(stock_df)):
    if partition_size_short == 0:
        if stock_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = stock_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (stock_df['Close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(stock_df['Open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

C:\Users\Admin\AppData\Local\Temp\ipykernel_28252\2404176544.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if stock_df['long_condition'][i] == 1:
C:\Users\Admin\AppData\Local\Temp\ipykernel_28252\2404176544.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stop_loss_long = stock_df['five_day_min_low'][i]
C:\Users\Admin\AppData\Local\Temp\ipykernel_28252\2404176544.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (sto

Start                         2023-02-17 00:00:00
End                           2023-12-29 00:00:00
Period                          214 days 00:00:00
Start Value                                 100.0
End Value                               132.72989
Total Return [%]                         32.72989
Benchmark Return [%]                    -7.079646
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         22.68606
Max Drawdown Duration           110 days 00:00:00
Total Trades                                   37
Total Closed Trades                            36
Total Open Trades                               1
Open Trade PnL                           4.213647
Win Rate [%]                                 50.0
Best Trade [%]                          22.222222
Worst Trade [%]                        -12.429379
Avg Winning Trade [%]                    5.206566
Avg Losing Trade [%]                    -3.817425


"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

In [4]:
pf.plot().show()